<span style="font-size:large;"> <strong>B</strong>roadcastify <strong>AR</strong>chive <strong>T</strong>oolkit</span>
```
 |\/\/\/|  +------------------------+
 |      |  | Eat my shortwave, man. |
 |      |  +------------------------+
 | (o)(o)    /
 C      _)  /
  | ,___|  
  |   /    
 /____\    
/      \
```

<sub><sup>Artwork source:
https://www.asciiart.eu/cartoons/simpsons</sup></sub>

## Imports & Declarations

In [1]:
import pdb
import os
import pandas as pd
import re
import requests

from datetime import datetime, timedelta
from IPython.display import clear_output
from time import time as timer

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support import wait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## For abandoned approaches
# import sys
# import time
# from PyQt5.QtWidgets import QApplication
# from PyQt5.QtCore import QUrl
# from PyQt5.QtWebEngineWidgets import QWebEngineView


In [2]:
REQUEST_WAIT_SECS = 3
# DATA_PATH = '../⁨data/⁨test_data/test_mp3⁩/' <-- This one has NPC from copying from Finder...
    # only discovered b/c google's search for this page https://www.reddit.com/r/learnpython/comments/aafp0x/how_should_i_open_a_txt_file_that_downloaded/
    # had the npc's encoded in the search result (https://www.google.com/search?q=%5BErrno+2%5D+No+such+file+or+directory%3A+%27..%2F%5Cu2068&rlz=1C5CHFA_enUS845US845&oq=%5BErrno+2%5D+No+such+file+or+directory%3A+%27..%2F%5Cu2068&aqs=chrome..69i57.4788j0j7&sourceid=chrome&ie=UTF-8)

DATA_PATH = '../data/'
TEST_MP3_OUT_PATH = DATA_PATH + 'test_data/test_mp3/'

TEST_FEED_ID = '18812'

USERNAME = 'cwchiu'
PASSWORD = 'datascientists'

## Notebook-level functions

In [3]:
def time_to_hhmm(s):
    # Converts a string representing a time in HH:MM AM/PM format to a string in 24-hr HHMM
    return ''.join(str(datetime.strptime(s, '%I:%M %p')).split(' ')[-1].split(':')[:2])

    # More details, since it's a one-line and this isn't freaking codewars:
        # strptime converts the string to datetime 
            # see https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
            # and https://stackoverflow.com/questions/19229190/convert-12-hour-into-24-hour-times
        # first split separates YYYY-MM-DD from HH:MM
        # second split gets rid of the colon between HH & MM
        # join puts HHMM together

In [4]:
def parse_archive_times_table(archive_page_soup):
    """
    Generates a list of Broadcastify archive file information from
    the `archiveTimes` table on a feed's archive page. Each item in
    the list is a list of two elements:
        - Date and end time of the transmission in the format YYYYMMDD-HHMM,
          on a 24-hour clock
        - The unique ID for the file, which can be used to find the file's
          individual download page
         
    Parameters
    ----------
    archive_page_soup : bs4.BeautifulSoup
        A BeautifulSoup object containing the feed archive page source code, 
        e.g. from https://m.broadcastify.com/archives/feed/[feed_id]
        
        
    """
    # Set up a blank list to return
    files_to_download = []
    
    # Isolate the `archive_times` table body
    archive_times = archive_page_soup.find('table', attrs={'id': 'archiveTimes'}).find('tbody')
    
    # Find the date of transmission of the archived files
    archive_date = get_archive_date(archive_page_soup)
    
    # Loop through all rows of the table
    for row in archive_times.find_all('tr'):
#         file = []
#         file_info = []

        # Grab the end time, contained in the row's second <td> tag
        file_end_time = time_to_hhmm(row.find_all('td')[1].text) 
        pdb.set_trace()
        # Represent the date & end time of the file as YYYYMMDD-HHMM
        file_end_date_time = '-'.join([archive_date, file_end_time])

        # Grab the file ID
        file_id = row.find('a')['href'].split('/')[-1]

        # Put the file date/time and URL leaf (as a list) into the list
        files_to_download.append([file_end_date_time, file_id])

    return files_to_download

In [5]:
def get_archive_date(archive_page_soup):
    MONTHS = ['','January', 'February', 'March',
          'April', 'May', 'June',
          'July', 'August', 'September',
          'October', 'November', 'December']
    
    # Extract the day, month, and year of the data displayed on the page
    day = archive_page_soup.find('td', {'class': 'active day'}).text
    month, year = archive_page_soup.find('th', {'class': 'datepicker-switch'}).text.split()
    
    # Format the date as YYYYMMDD
    formatted_date = str(year) + str(MONTHS.index(month)).zfill(2) + day.zfill(2)

    return formatted_date

In [6]:
def get_mp3_path(download_page_soup):
    # Get the filepath for the mp3 archive
    return download_page_soup.find('a', {'href': re.compile('.mp3')}).attrs['href']

In [7]:
def courtesy_wait(last_hit, wait_time=REQUEST_WAIT_SECS):
    return time.time() - last_hit >= wait_time

----

## File Retrieval

### Scrape feed archive page with `selenium`

Need to use browser emulation because the page elements we need are rendered in JavaScript after the page loads.

In [8]:
# Instantiate a browser
browser = webdriver.Chrome('../assets/chromedriver')

# Open the feed archive page
browser.get('https://m.broadcastify.com/archives/feed/' + TEST_FEED_ID)

## Wait for page to render
element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "cursor-link")))

# Capture page content as a BSoup
archive_page_soup = BeautifulSoup(browser.page_source, 'lxml')

# Quit the browser
browser.quit()

In [9]:
archive_page_soup

<html lang="en"><head>
<!-- Title -->
<title>Atlanta Police Zone 5 and Fire Dispatch Audio Archives</title>
<!-- Required Meta Tags Always Come First -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<!-- Favicon -->
<link href="https://s.broadcastify.com/i/favicon.ico" rel="shortcut icon"/>
<link href="https://s.broadcastify.com/i/apple-touch-icon-v2.png" rel="apple-touch-icon"/>
<!-- Google Fonts -->
<link href="//fonts.googleapis.com/css?family=Poppins:300,400,500,600,700" rel="stylesheet"/>
<!-- CSS Implementing Plugins -->
<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.9.0/css/all.min.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/animate.css/animate.min.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/hs-megamenu/src/hs.megamenu.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/ven

In [21]:
archive_page_soup.find_all('a', 
                           {'href':re.compile('/download/')
                           })[0]

<a class="text-dark fa fa-download float-right" download="" href="/archives/download/774729154" title="Download audio"></a>

In [16]:
current_first_id = archive_page_soup.find_all('a', 
                                              {'href':re.compile('/download/')
                                              })[0].attrs['href'].split('/')[-1]

In [20]:
current_first_id

'774729154'

In [36]:
# Instantiate a browser
browser = webdriver.Chrome('../assets/chromedriver')

# Open the feed archive page
browser.get('https://m.broadcastify.com/archives/feed/' + TEST_FEED_ID)

## Wait for page to render
element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "cursor-link")))

element = browser.find_element_by_xpath(f"//a[@class='text-dark fa fa-download float-right'"
                                        f"and starts-with(@href, '/archives/download')]")

print(element.get_attribute('href').split('/')[-1])

#//a[@class='text-dark fa fa-download float-right' and starts-with(@href, '/archives/download')]
#//a[@class='bf' and starts-with(@href, '/book/')]


# //Element[@attribute1="abc" and @attribute2="xyz" and text()="Data"]
#'//a[@href.contains(text(), '774729154')]'
#//a[text()='text_i_want_to_find']/@href
# Capture page content as a BSoup
# archive_page_soup = BeautifulSoup(browser.page_source, 'lxml')

# Quit the browser
# browser.quit()

774734412


In [35]:
element.get_attribute('href').split('/')[-1]

'774734412'

In [ ]:
String bodyText = driver.findElement(By.tagName("body")).getText();
Assert.assertTrue("Text not found!", bodyText.contains(text));

### Parse the page's `archiveTimes` table

In [14]:
archive_info = parse_archive_times_table(archive_page_soup)

> <ipython-input-13-6b580d406e7f>(37)parse_archive_times_table()
-> file_end_date_time = '-'.join([archive_date, file_end_time])
(Pdb) file_end_date
*** NameError: name 'file_end_date' is not defined
(Pdb) file_end_time
'2052'
(Pdb) row.find_all('td')[1].text
'08:52 PM'
(Pdb) row.find_all('td')
[<td>08:22 PM</td>, <td>08:52 PM<a class="text-dark fa fa-download float-right" download="" href="/archives/download/774527084" title="Download audio"></a></td>]
(Pdb) continue
> <ipython-input-13-6b580d406e7f>(35)parse_archive_times_table()
-> pdb.set_trace()
(Pdb) c
> <ipython-input-13-6b580d406e7f>(37)parse_archive_times_table()
-> file_end_date_time = '-'.join([archive_date, file_end_time])
(Pdb) continue
> <ipython-input-13-6b580d406e7f>(35)parse_archive_times_table()
-> pdb.set_trace()
(Pdb) exit


BdbQuit: 

In [11]:
archive_info

[['20190729-2052', '774527084'],
 ['20190729-2022', '774520118'],
 ['20190729-1952', '774514488'],
 ['20190729-1922', '774506897'],
 ['20190729-1852', '774500371'],
 ['20190729-1823', '774493320'],
 ['20190729-1753', '774488161'],
 ['20190729-1706', '774477203'],
 ['20190729-1653', '774474653'],
 ['20190729-1623', '774468287'],
 ['20190729-1553', '774459572'],
 ['20190729-1524', '774454646'],
 ['20190729-1454', '774446543'],
 ['20190729-1424', '774440091'],
 ['20190729-1354', '774433700'],
 ['20190729-1324', '774426456'],
 ['20190729-1254', '774420052'],
 ['20190729-1225', '774412822'],
 ['20190729-1155', '774406710'],
 ['20190729-1125', '774400760'],
 ['20190729-1055', '774394136'],
 ['20190729-1025', '774387434'],
 ['20190729-0955', '774380685'],
 ['20190729-0926', '774373462'],
 ['20190729-0856', '774365895'],
 ['20190729-0826', '774360576'],
 ['20190729-0756', '774353156'],
 ['20190729-0726', '774345816'],
 ['20190729-0656', '774339017'],
 ['20190729-0627', '774333104'],
 ['2019072

### Get `mp3` paths for archived files

Login required to access pages in `/archives/id/`. 

Consider re-implementing with a `requests.Session`...not sure if it would be faster or by how much...

In [156]:
# Instantiate a browser
browser = webdriver.Chrome('../assets/chromedriver')

# Get the first page
browser.get('https://m.broadcastify.com/archives/id/' + archive_info[0][1])
last_page_request_time = time.time()

# Log in so we can download files
## Store the fields for username + password
username = browser.find_element_by_id("signinSrEmail") 
password = browser.find_element_by_id("signinSrPassword")

## Type username + password, and hit "enter"
username.send_keys(USERNAME)
password.send_keys(PASSWORD)
password.send_keys(Keys.RETURN)

## Wait for login to complete
browser.implicitly_wait(2)

# Get the filepath for the mp3 archive
archive_info[0].append(get_mp3_path(BeautifulSoup(browser.page_source, 'lxml')))

for row in archive_info[1:11]:
   # Wait until some time has passed, out of courtesy
    while not courtesy_wait(last_page_request_time): pass

    # Get the next archive page, recording the time
    browser.get('https://m.broadcastify.com/archives/id/' + row[1])
    last_page_request_time = time.time()
    
    # Get the filepath for the mp3 archive
    row.append(get_mp3_path(BeautifulSoup(browser.page_source, 'lxml')))

# Quit the browser
browser.quit()

In [157]:
archive_info

[['20190725-1924',
  '773214787',
  'http://garchives1.broadcastify.com/18812/20190725/201907251754-187417-18812.mp3'],
 ['20190725-1854',
  '773208421',
  'http://garchives1.broadcastify.com/18812/20190725/201907251724-382249-18812.mp3'],
 ['20190725-1824',
  '773202177',
  'http://garchives1.broadcastify.com/18812/20190725/201907251654-790986-18812.mp3'],
 ['20190725-1754',
  '773194340',
  'http://garchives1.broadcastify.com/18812/20190725/201907251624-881908-18812.mp3'],
 ['20190725-1725',
  '773188378',
  'http://garchives1.broadcastify.com/18812/20190725/201907251555-210863-18812.mp3'],
 ['20190725-1655',
  '773180493',
  'http://garchives1.broadcastify.com/18812/20190725/201907251525-847590-18812.mp3'],
 ['20190725-1625',
  '773175133',
  'http://garchives1.broadcastify.com/18812/20190725/201907251455-596093-18812.mp3'],
 ['20190725-1555',
  '773168047',
  'http://garchives1.broadcastify.com/18812/20190725/201907251425-507163-18812.mp3'],
 ['20190725-1525',
  '773162347',
  'htt

### Download `.mp3` files

In [63]:
# https://markhneedham.com/blog/2018/07/15/python-parallel-download-files-requests/

def fetch_mp3(entry):
    path, uri = entry
    
    if not os.path.exists(path):
        r = requests.get(uri, stream=True)
        if r.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
    return path

In [158]:
start = timer()
last_page_request_time = time.time() - REQUEST_WAIT_SECS

for file in archive_info:
    file_date = file[0]
    file_id = file[1]
    url = file[2]
    
    # Build the filename we'll store the downloaded .mp3 under
    file_name = TEST_MP3_OUT_PATH + '-'.join([TEST_FEED_ID, file_date]) + '.mp3'

    print(f'Downloading {archive_info.index(file) + 1} of {len(archive_info)}')
    print(f'\tfrom {url}')
    print(f'\tto {file_name}')
    
    # Wait until some time has passed, out of courtesy
    while not courtesy_wait(last_page_request_time): pass

    fetch_mp3([file_name, url])
    last_page_request_time = time.time()
    clear_output(wait=True)
        # h/t @schmitty
    
print('**** Downloads complete! ****')
print(f'Elapsed Time: {round(timer() - start, 4)} seconds.')

	from http://garchives1.broadcastify.com/18812/20190725/201907251624-881908-18812.mp3
	to ../data/test_data/test_mp3/18812-20190725-1754.mp3


KeyboardInterrupt: 

## Calendar Manipulation

In [159]:
active_day = archive_page_soup.find('td', {'class': 'active day'})
active_day

<td class="active day">25</td>

In [211]:
# Instantiate a browser
browser = webdriver.Chrome('../assets/chromedriver')

# Open the feed archive page
browser.get('https://m.broadcastify.com/archives/feed/' + TEST_FEED_ID)

## Wait for page to render
element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "cursor-link")))

calendar_day = browser.find_element_by_xpath("//td[@class='day' and contains(text(),'1')]")
    # https://stackoverflow.com/questions/2009268/how-to-write-an-xpath-query-to-match-two-attributes
calendar_day.click()

archive_page_soup = BeautifulSoup(browser.page_source, 'lxml')

# Quit the browser
# browser.quit()

In [212]:
calendar_day.text

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=75.0.3770.142)


In [175]:
archive_page_soup

<html lang="en"><head>
<!-- Title -->
<title>Atlanta Police Zone 5 and Fire Dispatch Audio Archives</title>
<!-- Required Meta Tags Always Come First -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<!-- Favicon -->
<link href="https://s.broadcastify.com/i/favicon.ico" rel="shortcut icon"/>
<link href="https://s.broadcastify.com/i/apple-touch-icon-v2.png" rel="apple-touch-icon"/>
<!-- Google Fonts -->
<link href="//fonts.googleapis.com/css?family=Poppins:300,400,500,600,700" rel="stylesheet"/>
<!-- CSS Implementing Plugins -->
<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.9.0/css/all.min.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/animate.css/animate.min.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/hs-megamenu/src/hs.megamenu.css" rel="stylesheet"/>
<link href="https://s.broadcastify.com/front-v2.8.0/assets/ven

In [176]:
calendar = archive_page_soup.find('table', {'class': 'table-condensed'})

In [177]:
calendar

<table class="table-condensed"><thead><tr><th class="datepicker-title" colspan="7" style="display: none;"></th></tr><tr><th class="prev" style="visibility: visible;">«</th><th class="datepicker-switch" colspan="5">July 2019</th><th class="next" style="visibility: hidden;">»</th></tr><tr><th class="dow">Su</th><th class="dow">Mo</th><th class="dow">Tu</th><th class="dow">We</th><th class="dow">Th</th><th class="dow">Fr</th><th class="dow">Sa</th></tr></thead><tbody><tr><td class="old day">30</td><td class="day">1</td><td class="day">2</td><td class="day">3</td><td class="day">4</td><td class="day">5</td><td class="day">6</td></tr><tr><td class="day">7</td><td class="day">8</td><td class="day">9</td><td class="day">10</td><td class="day">11</td><td class="day">12</td><td class="day">13</td></tr><tr><td class="day">14</td><td class="day">15</td><td class="day">16</td><td class="day">17</td><td class="day">18</td><td class="day">19</td><td class="day">20</td></tr><tr><td class="day">21</td

In [213]:
calendar.find('th', {'class': 'datepicker-switch'}).text.split(' ')

['July', '2019']

In [155]:
calendar.find_all('td', limit=5)

# use new_tag to add 31, 32, 33 to test prior_month_last_day list comp

[<td class="old day">30</td>,
 <td class="day">1</td>,
 <td class="day">2</td>,
 <td class="day">3</td>,
 <td class="day">4</td>]

In [242]:
days_on_calendar = calendar.find_all('td')
type(days_on_calendar)

bs4.element.ResultSet

In [218]:
days_copy = days_on_calendar.copy()

In [237]:
days_copy[5]['class'] = ['active', 'day']

In [241]:
type(days_copy)

list

In [239]:
def parse_month_min_day(days_on_calendar):
    # Parse the lowest valid day in the displayed month
    disabled_found = False
    for day in days_on_calendar:
        print(day['class'])

        if day['class'][0] == 'disabled':
            disabled_found = True
        elif day['class'][0] in 'day active'.split():
            return int(day.text)
        elif day['class'][0] != 'old' and disabled_found:
            return int(day.text)

    return None

In [240]:
parse_month_min_day(days_copy)

['old', 'disabled', 'day']
['disabled', 'day']
['disabled', 'day']
['disabled', 'day']
['disabled', 'day']
['active', 'day']


5

In [216]:
rev = [day['class'][0] for day in days_on_calendar][::-1]
print(rev)

['new', 'new', 'new', 'new', 'new', 'new', 'new', 'new', 'new', 'new', 'disabled', 'disabled', 'disabled', 'disabled', 'disabled', 'disabled', 'active', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'day', 'old']


In [188]:
min([day['class'][0] for day in days_on_calendar].index('day'),
    [day['class'][0] for day in days_on_calendar].index('active'),
    [day['class'][0] for day in days_on_calendar].index('disabled'))

1

In [181]:
days_on_calendar

[<td class="old day">30</td>,
 <td class="day">1</td>,
 <td class="day">2</td>,
 <td class="day">3</td>,
 <td class="day">4</td>,
 <td class="day">5</td>,
 <td class="day">6</td>,
 <td class="day">7</td>,
 <td class="day">8</td>,
 <td class="day">9</td>,
 <td class="day">10</td>,
 <td class="day">11</td>,
 <td class="day">12</td>,
 <td class="day">13</td>,
 <td class="day">14</td>,
 <td class="day">15</td>,
 <td class="day">16</td>,
 <td class="day">17</td>,
 <td class="day">18</td>,
 <td class="day">19</td>,
 <td class="day">20</td>,
 <td class="day">21</td>,
 <td class="day">22</td>,
 <td class="day">23</td>,
 <td class="day">24</td>,
 <td class="active day">25</td>,
 <td class="disabled day">26</td>,
 <td class="disabled day">27</td>,
 <td class="disabled day">28</td>,
 <td class="disabled day">29</td>,
 <td class="disabled day">30</td>,
 <td class="disabled day">31</td>,
 <td class="new disabled day">1</td>,
 <td class="new disabled day">2</td>,
 <td class="new disabled day">3</td>

In [170]:
int([day.text for day in days_on_calendar
                if (day['class'][0] == 'disabled')][0]) - 1

25

In [161]:
days_to_view = [day.text for day in days_on_calendar
                if (day['class'][0] == 'day') or
                   (day['class'][0] == 'active')][::-1]

In [165]:
[day.text for day in days_on_calendar
                if (day['class'][0] == 'active')]

['25']

In [162]:
days_to_view

['25',
 '24',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '9',
 '8',
 '7',
 '6',
 '5',
 '4',
 '3',
 '2',
 '1']

In [149]:
# prior_month_last_day = 
[day.text for day in days_on_calendar
                if (day['class'][0] == 'old')][::-1]

['30']

----

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

##  

## Previous attempts

### Scrape `.mp3` download page with `requests`

In [19]:
login_data = {
    'username': 'cwchiu',
    'password': 'datascientists',
    'action': 'auth',
    'redirect': '/'
}

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) ' +
                  'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/' +
                  '75.0.3770.142 Safari/537.36'
}

login_url = 'https://m.broadcastify.com/login/'

In [25]:
# h/t to Indian Pythonista for the info about request.Session
    # https://www.youtube.com/watch?v=fmf_y8zpOgA
start_time = timer()

with requests.Session() as s:
    r = s.post(login_url, data=login_data, headers=headers)
    r = s.get('https://m.broadcastify.com/archives/id/772540078')
    soup = BeautifulSoup(r.text, 'lxml')
    a_list = soup.find('a', {'href': re.compile('.mp3')})

print(timer() - start_time)

0.6505601406097412


In [26]:
a_list.attrs['href']

'http://garchives1.broadcastify.com/18812/20190723/201907231540-907553-18812.mp3'

In [27]:
r.text

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <!-- Title -->\n    <title>Broadcastify</title>\n\n    <!-- Required Meta Tags Always Come First -->\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n\n    <!-- Favicon -->\n    <link rel="shortcut icon" href="https://s.broadcastify.com/i/favicon.ico">\n    <link rel="apple-touch-icon" href="https://s.broadcastify.com/i/apple-touch-icon-v2.png" />\n\n    <!-- Google Fonts -->\n    <link href="//fonts.googleapis.com/css?family=Poppins:300,400,500,600,700" rel="stylesheet">\n\n    <!-- CSS Implementing Plugins -->\n    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.9.0/css/all.min.css">\n    <link rel="stylesheet" href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/animate.css/animate.min.css">\n    <link rel="stylesheet" href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/hs-megamenu/src/hs.megamenu.css">\n    <link r

### Scrape feed archive page with `requests`

**Atlanta Police Zone 5 and Fire Dispatch**

Broadcasting Atlanta PD Zone 5 dispatch and Atlanta Fire and Rescue dispatch from Uniden Home Patrol and Windows 10 laptop. Limited to dispatch only. Adding allowed tac sub-channels will talk-over the dispatch channels.

https://www.broadcastify.com/listen/feed/18812

Feed archives = https://m.broadcastify.com/archives/feed/18812

In [22]:
# h/t to Indian Pythonista for the info about request.Session
    # https://www.youtube.com/watch?v=fmf_y8zpOgA
start_time = timer()

with requests.Session() as s:
    r = s.post(login_url, data=login_data, headers=headers)
    r = s.get('https://m.broadcastify.com/archives/feed/18812')
#     while time.time() - start_time <= 5: pass
    soup = BeautifulSoup(r.text, 'html5lib')
    a_list = soup.find_all('a', {'title': 'Download audio'})
    
print(timer() - start_time)

0.4976658821105957


In [23]:
a_list

[]

In [24]:
soup

<!DOCTYPE html>
<html lang="en"><head>
    <!-- Title -->
    <title>Atlanta Police Zone 5 and Fire Dispatch Audio Archives</title>

    <!-- Required Meta Tags Always Come First -->
    <meta charset="utf-8"/>
    <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>

    <!-- Favicon -->
    <link href="https://s.broadcastify.com/i/favicon.ico" rel="shortcut icon"/>
    <link href="https://s.broadcastify.com/i/apple-touch-icon-v2.png" rel="apple-touch-icon"/>

    <!-- Google Fonts -->
    <link href="//fonts.googleapis.com/css?family=Poppins:300,400,500,600,700" rel="stylesheet"/>

    <!-- CSS Implementing Plugins -->
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.9.0/css/all.min.css" rel="stylesheet"/>
    <link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/animate.css/animate.min.css" rel="stylesheet"/>
    <link href="https://s.broadcastify.com/front-v2.8.0/assets/vendor/hs-megamenu/src/hs.megamenu.css" rel="

### Scrape feed archive page with `QWebEngineView` custom class

In [ ]:
class Render(QWebEngineView):
        def __init__(self, url):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            #self.setHtml(html)
            self.load(QUrl(url))
            self.app.exec_()

        def _loadFinished(self, result):
            # This is an async call, you need to wait for this
            # to be called before closing the app
            self.page().toHtml(self._callable)

        def _callable(self, data):
            self.html = data
            # Data has been stored, it's safe to quit the app
            self.app.quit()

In [ ]:
def render(source_url):
    """Fully render HTML, JavaScript and all."""
    return Render(source_url).html

In [ ]:
url="https://m.broadcastify.com/archives/feed/18812"
print(render(url))

In [ ]:
# import os

# for file in archive_info[0]:
#     file_id = file[1]
#     url = file[2]
    
#     print(f'Downloading {file_id}')
#     os.system(f'wget {url}')
    
    
#     'wget --http-user={USERNAME} --http-password={PASSWORD} {url}'
#     '--user-agent'